In [1]:
#!pip install PeptideBuilder
#!pip install nglview
#!pip install Bio
#!pip install numpy

from io import StringIO
import Bio
from Bio.PDB import PDBIO
import PeptideBuilder
import nglview as nv
import numpy as np

## Peptide construction


The goal of this tutorial is to make a peptide with any amino acid sequence and shape you like. We'll show you how to do it by building a couple of different peptides:

* Example 1: A simple alpha helix made of 10 alanines.
* Example 2: A simple beta strand made of 10 alanines.
* Example 3: A more interesting peptide (several different aminoacids, two different secondary structures.

Then, we can get creative and add any weird atom you want to any peptide:

* Example 4: Add a gold atom to a peptide
* Example 5: Add CAPs to a peptide

The picture you see at the top of this page is an example of something you will be able to build after going through this tutorial.

### Example 1: simple alpha helix made of 10 alanines

Peptide Builder operates by creating a geometry object for every amino acid you want to include in your chain. Each geometry object stores the angle phi and psi, that define how the amino acid is oriented. In the case of a perfect alpha helix, those angles should be -57.8 and -47.

These geometry objects are then combined to form a single peptide object, representing the complete structure of the peptide. 

Once assembled, you can export this structure to a PDB file for further analysis or visualization.

But this explanation might sound too abstract. It's better to look at concrete code that does all of that. 


In [2]:

# Define aminoacids
sequence = "AAAAAAAAAA"


# Add the rest of the amino acids to the peptide
for i in range(0,len(sequence)):

    #get current aminoacid letter code
    a=sequence[i]

    #define the geometry of the current aminoacid
    current_aa_geometry = PeptideBuilder.Geometry.geometry(a)
    current_aa_geometry.phi = -57.8
    current_aa_geometry.psi_im1 = -47.0 #angle of the immediately preceding residue (where "im1" denotes "i minus 1")

    #insert the current aminoacid in peptide
    if i == 0:
        peptide1 = PeptideBuilder.initialize_res(current_aa_geometry) # Initialize the peptide with the first amino acid
    else:
        PeptideBuilder.add_residue(peptide1, current_aa_geometry) # Add current aminoacid to previouly constructed peptide

# Save file
io = PDBIO()
io.set_structure(peptide1)
io.save("helical_alanines.pdb")


Now we should be able to see the PDB file we created. We'll use NGViewer for that. (While doing it, please notice that sometimes NGViewer just shows a blank image. In those cases, just close and reopen the Jupyter notebook, it should work then.)

In [3]:
view = nv.show_file('helical_alanines.pdb')
view.clear()
view.add_representation('ball+stick', selection='protein')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

But there is a catch here: the peptide has no hydrogens. To add hydrogens, we will use GROMACS. This will also convert the PDB file into .gro and .top files. (This step requires, of course, that you have GROMACS already installed in your local environment.)

In [4]:
!gmx pdb2gmx -f helical_alanines.pdb -o helical_alanines.gro -p helical_alanines.top -i helical_alanines.posres.itp -ignh -water tip3p -ff charmm27



               :-) GROMACS - gmx pdb2gmx, 2023.4-conda_forge (-:

Executable:   /home/hrigitano/miniconda3/envs/bio/bin.AVX2_256/gmx
Data prefix:  /home/hrigitano/miniconda3/envs/bio
Working dir:  /data3/henrique/templates/peptide_construction
Command line:
  gmx pdb2gmx -f helical_alanines.pdb -o helical_alanines.gro -p helical_alanines.top -i helical_alanines.posres.itp -ignh -water tip3p -ff charmm27

Using the Charmm27 force field in directory charmm27.ff

going to rename charmm27.ff/aminoacids.r2b
Opening force field file /home/hrigitano/miniconda3/envs/bio/share/gromacs/top/charmm27.ff/aminoacids.r2b

going to rename charmm27.ff/rna.r2b
Opening force field file /home/hrigitano/miniconda3/envs/bio/share/gromacs/top/charmm27.ff/rna.r2b
Reading helical_alanines.pdb...
Read '', 50 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 10 residues with 50 atoms

  chain  #res #atoms

  1 'A'    10   

If we visualize the .gro file created from the original PDB, we'll now find hydrogens.

In [5]:
#view
view = nv.show_file('helical_alanines.gro')
view.clear()
view.add_representation('ball+stick', selection='protein')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

## Example 2: A simple beta strand made of 10 alanines

The code is the same as the previous one that created the helical alanine. The only difference is the setup of the phi and psi angles. In the case of a beta strand, those angles should be -120 and 120.

In [6]:

# Define aminoacids
sequence = "AAAAAAAAAA"


# Add the rest of the amino acids to the peptide
for i in range(0,len(sequence)):

    #get current aminoacid letter code
    a=sequence[i]

    #define the geometry of the current aminoacid
    current_aa_geometry = PeptideBuilder.Geometry.geometry(a)
    current_aa_geometry.phi = -120
    current_aa_geometry.psi_im1 = 120 #angle of the immediately preceding residue (where "im1" denotes "i minus 1")

    #insert the current aminoacid in peptide
    if i == 0:
        peptide2 = PeptideBuilder.initialize_res(current_aa_geometry) # Initialize the peptide with the first amino acid
    else:
        PeptideBuilder.add_residue(peptide2, current_aa_geometry) # Add current aminoacid to previouly constructed peptide

# Save file
io = PDBIO()
io.set_structure(peptide2)
io.save("extended_alanines.pdb")





The visualisation code is exactly the same

In [7]:
#view
view = nv.show_file('extended_alanines.pdb')
view.clear()
view.add_representation('ball+stick', selection='protein')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

Again, the molecule doesn't have any hydrogens. But you already know the drill: You have to add them using GROMACS. The hydrogens will be present in the output .gro file.

In [8]:
!gmx pdb2gmx -f extended_alanines.pdb -o extended_alanines.gro -p extended_alanines.top -i extended_alanines.posres.itp -ignh -water tip3p -ff charmm27


               :-) GROMACS - gmx pdb2gmx, 2023.4-conda_forge (-:

Executable:   /home/hrigitano/miniconda3/envs/bio/bin.AVX2_256/gmx
Data prefix:  /home/hrigitano/miniconda3/envs/bio
Working dir:  /data3/henrique/templates/peptide_construction
Command line:
  gmx pdb2gmx -f extended_alanines.pdb -o extended_alanines.gro -p extended_alanines.top -i extended_alanines.posres.itp -ignh -water tip3p -ff charmm27

Using the Charmm27 force field in directory charmm27.ff

going to rename charmm27.ff/aminoacids.r2b
Opening force field file /home/hrigitano/miniconda3/envs/bio/share/gromacs/top/charmm27.ff/aminoacids.r2b

going to rename charmm27.ff/rna.r2b
Opening force field file /home/hrigitano/miniconda3/envs/bio/share/gromacs/top/charmm27.ff/rna.r2b
Reading extended_alanines.pdb...
Read '', 50 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 10 residues with 50 atoms

  chain  #res #atoms

  1 'A'    

In [9]:
#view
view = nv.show_file('extended_alanines.gro')
view.clear()
view.add_representation('ball+stick', selection='protein')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

## Example 3 : A more interesting peptide

Now lets create peptide made of interesting amino acids.

For an interesting mix, we have, among others, serine (S) for potential phosphorylation sites, cysteine (C) for potential disulfide bond formation, histidine (H) which can be protonated or deprotonated depending on pH, and tryptophan (W) with a bulky ring that can be involved in protein-protein interactions.

More over lets try build a secondary structure made of a alpha helix of 10 residues, and then the structure changes into a beta strand. 

I don't know if such a peptide is stable enough to exist in real life. This is just to learn how to play with the angles.

In [10]:


# Define the aminoacids
sequence = "MCEHMSALEWALEHMSALEM"


# Construct the pepide up untill the 10th residue as a alpha helix
for i in range(0,11):

    #get current aminoacid letter code
    a=sequence[i]

    #define the geometry of the current aminoacid
    current_aa_geometry = PeptideBuilder.Geometry.geometry(a)
    current_aa_geometry.phi = -57.8
    current_aa_geometry.psi_im1 = -47.0 #angle of the immediately preceding residue (where "im1" denotes "i minus 1")

    #insert the current aminoacid in peptide
    if i == 0:
        peptide3 = PeptideBuilder.initialize_res(current_aa_geometry) # Initialize the peptide with the first amino acid
    else:
        PeptideBuilder.add_residue(peptide3, current_aa_geometry) # Add current aminoacid to previouly constructed peptide




# Add the rest of the amino acids to the peptide, but now as a beta strand
for i in range(10,len(sequence)):

    #get current aminoacid letter code
    a=sequence[i]

    #define the geometry of the current aminoacid
    current_aa_geometry = PeptideBuilder.Geometry.geometry(a)
    current_aa_geometry.phi = -120
    current_aa_geometry.psi_im1 = 120 #angle of the immediately preceding residue (where "im1" denotes "i minus 1")

    #insert the current aminoacid in peptide
    PeptideBuilder.add_residue(peptide3, current_aa_geometry) # Add current aminoacid to previouly constructed peptide







# Save file
io = PDBIO()
io.set_structure(peptide3)
io.save("interesting.pdb")


In [2]:
#view
view = nv.show_file('interesting.pdb')
view.clear()
view.add_representation('ball+stick', selection='protein')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

## Example 4: Add a gold atom to a peptide


We star by building a simple peptide (6 residue alanine). We already know how to do that.

In [23]:
# Define aminoacids
sequence = "AAAAAA"


# Add the rest of the amino acids to the peptide
for i in range(0,len(sequence)):

    #get current aminoacid letter code
    a=sequence[i]

    #define the geometry of the current aminoacid
    current_aa_geometry = PeptideBuilder.Geometry.geometry(a)
    current_aa_geometry.phi = -57.8
    current_aa_geometry.psi_im1 = -47.0 #angle of the immediately preceding residue (where "im1" denotes "i minus 1")

    #insert the current aminoacid in peptide
    if i == 0:
        peptide4 = PeptideBuilder.initialize_res(current_aa_geometry) # Initialize the peptide with the first amino acid
    else:
        PeptideBuilder.add_residue(peptide4, current_aa_geometry) # Add current aminoacid to previouly constructed peptide

# Save file
io = PDBIO()
io.set_structure(peptide4)
io.save("helical_6alanine.pdb")

In [24]:
#view
view = nv.show_file('helical_6alanine.pdb')
view.clear()
view.add_representation('ball+stick')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

Now, to add weird molecules to it, we have to make use of the Bio.PDB.Atom.Atom object. It alows us to put any atom we want in any position. To succeed, what we have to to is to know the correct coordinates to put it. For example, to put a gold in the tip (N-terminus) first we have to obtain the coordenates of the N atom on that N-terminus, and then set the gold atom at some distance of that coordenate. We can set any distance we want, but it would be wise to set a distance of 2.1, because thats the real distance between N and Au in real molecules.



In [25]:
def add_gold(peptide):

    #obtain the list of all residues. remember that the peptide object can have several models (we get the first, with id 0) and a model can have several chains (we get the only one, named A)
    l_residues =  list(peptide[0]['A'].get_residues()) 

    # Get the first residue in the chain
    first_residue = l_residues[0]  
    # Obtain the coordenates of the nitrogen in that first residue
    x, y, z  = first_residue['N'].get_coord() 

    # Add the gold Atom, using the coordenates we just obtained
    acetyl_c1 = Bio.PDB.Atom.Atom("1ACE", [x - 2.1, y, z], 0.0, 1.0, ' ', '1ACE ', 1001, 'AU')


    # Add the acetyl atoms to the residue
    first_residue.add(acetyl_c1)

# Usage
add_gold(peptide4)

# Save file
io = PDBIO()
io.set_structure(peptide4)
io.save("helical_6alanine_with_gold.pdb")

Look at the gold atom on the tip! 

In [26]:
#view
view = nv.show_file('helical_6alanine_with_gold.pdb')
view.clear()
view.add_representation('ball+stick')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

But you have to be aware that this tool don juge what you do, it just obeys your orders without any questioning. This might be good, because you can actually do whatever you want. You can place any atoms at any coordenates. But in some cases that might be problematic, because you creation might be completely unrealistic and unstable .

## Example 5: Add CAPs to a peptide

After that exotic gold addition, let's do something more realistic: add an acetyl cap to the N-terminus. This is done frequently in the real pharmaceutical industry to eliminate partial charges at the N-terminus and make the peptide more stable under physiological conditions.

Once again, we begin by creating a peptide with 6 alanines.

In [29]:
# Define aminoacids
sequence = "AAAAAA"


# Add the rest of the amino acids to the peptide
for i in range(0,len(sequence)):

    #get current aminoacid letter code
    a=sequence[i]

    #define the geometry of the current aminoacid
    current_aa_geometry = PeptideBuilder.Geometry.geometry(a)
    current_aa_geometry.phi = -57.8
    current_aa_geometry.psi_im1 = -47.0 #angle of the immediately preceding residue (where "im1" denotes "i minus 1")

    #insert the current aminoacid in peptide
    if i == 0:
        peptide5 = PeptideBuilder.initialize_res(current_aa_geometry) # Initialize the peptide with the first amino acid
    else:
        PeptideBuilder.add_residue(peptide5, current_aa_geometry) # Add current aminoacid to previouly constructed peptide

# Save file
io = PDBIO()
io.set_structure(peptide5)
io.save("helical_6alanine.pdb")

In [30]:
#view
view = nv.show_file('helical_6alanine.pdb')
view.clear()
view.add_representation('ball+stick', selection='protein')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

Now we have to place that cap, but we should do that in a realistic way. That's kind of difficult to do, because 'Bio.PDB.Atom.Atom' requires x, y, and z coordinates. It's difficult to set those coordinates so to obey realistic distances and angles between atoms.

To solve that, let's create a function that give us the the x, y, and z coordinates, by giving as input some atoms present in the peptide, and also distances and angles using those atoms as reference.

Lets imagine how such a function could do the job.

The first thing the function should try to do is to place a new atom at a specified distance from a chosen atom in the peptide. But remember that only a distance from a single atom is not enough to define a unambiguous position of the new atom, as the new atom could be anywhere in a sphere around the reference atom. Thats why we also have to provide a second atom as input to the function. That will allow it to define the direction to which the new atom should go.

That could be enough in most cases. But let's be very professional and also allow setting angles of deviations from that direction. That will allow us to, for example, turn 45 degrees from that original direction. But then we hit another ambiguity; it's not possible to know in advance the direction of that angle. To solve this, we have to provide a third atom as input. That will define a plane. By doing this, we can provide the function with two angles: the angle within that plane, and the angle normal to that plane. Now we have the power to define the position in a very flexible way. To place, for example, a new atom on a hydrocarbon, where the carbons follow a zigzag pattern, one can do the following: find_new_atom_coord(last_atom, last_atom-1, last_atom-2, distance=1.5, angle_in_plane=45, angle_out_plane=0).

Here is the function:

In [31]:
def find_new_atom_coord(atom1, atom2, atom3, distance, angle_in_plane_deg, angle_to_plane_deg):
    """
    
    code in python to find those x y and z coordinates using as inputs: 
    a list with the coordinates of atom 1, 
    a list with the coordinates of atom 2, 
    a list with the coordinates of atom 3, 
    a distance from atom 1, 
    an angle (referenced from the line formed from the the line between atoms 1 and 2) within the plane formed by the 3 atoms, 
    and a second angle (referenced from the line formed from the the line between atoms 1 and 2) within the plane formed by the 3 atoms

    """

    
    # Convert angles from degrees to radians
    angle_in_plane = np.radians(angle_in_plane_deg)
    angle_to_plane = np.radians(angle_to_plane_deg)
    
    # Vectors between atoms
    v12 = np.array(atom1) - np.array(atom2)
    v13 = np.array(atom3) - np.array(atom1)
    
    # Normal to the plane defined by the three atoms
    normal = np.cross(v12, v13)
    normal_unit = normal / np.linalg.norm(normal)
    
    # Unit vector along atom1 to atom2
    v12_unit = v12 / np.linalg.norm(v12)
    
    # Vector in the plane perpendicular to v12
    in_plane_perpendicular = np.cross(normal, v12)
    in_plane_perpendicular_unit = in_plane_perpendicular / np.linalg.norm(in_plane_perpendicular)
    
    # New atom position calculation
    direction_vector = (np.cos(angle_in_plane) * v12_unit +
                        np.sin(angle_in_plane) * in_plane_perpendicular_unit)
    direction_vector_unit = direction_vector / np.linalg.norm(direction_vector)
    
    # Final position calculation
    final_vector = np.cos(angle_to_plane) * direction_vector_unit + np.sin(angle_to_plane) * normal_unit
    final_position = np.array(atom1) + distance * final_vector

    return final_position

Now, using that function, we can add the two carbons and the oxygen of the acetyl cap in a realistic position. Of course, you have to know what those realistic distances and angles are. But generally, this is something that chemists already know. In our case, the distance is 1.5 Å and 45 degrees from the previous atom.

In [32]:
def add_acetyl(peptide):
    
    chain = peptide[0]['A']
    l_residues = list(chain.get_residues())

    # Get the first residue in the chain and its coordinates
    first_residue = l_residues[0]
    coordsN = first_residue['N'].get_coord()
    coordsCA = first_residue['CA'].get_coord()
    coordsC = first_residue['C'].get_coord()

    # Create the first C of ACE
    coords = find_new_atom_coord(coordsN, coordsCA, coordsC, 1.5, -45, 0)
    acetyl_c1 = Bio.PDB.Atom.Atom("C", coords, 0.0, 1.0, ' ', 'C', 1001, 'C')

    # Create the second C, and the O of ACE
    coords = find_new_atom_coord(acetyl_c1.coord, coordsN, coordsCA, 1.5, +45, 0)
    acetyl_c2 = Bio.PDB.Atom.Atom("CH3", coords, 0.0, 1.0, ' ', 'CH3', 1002, 'C')
    coords = find_new_atom_coord(acetyl_c1.coord, coordsN, coordsCA, 1.5, -45, 0)
    acetyl_o = Bio.PDB.Atom.Atom("O", coords, 0.0, 1.0, ' ', 'O', 1003, 'O')

    #IMPORTANT: THE NAMES OF THE ATOMS IN ACE HAVE TO MATCH THE NAMES OF THE FORCEFIELD YOU WILL CHOSE IN THE FUTURE
    #THE NAMES "CH3", "C" AND "O" COMM FROM "CHARMM27". THEY CAN BE FOUND IN THE FILE "aminoacids.hdb" 

    # Create the new ACE residue
    ace_residue = Bio.PDB.Residue.Residue((' ', 1, ' '), 'ACE', '    ')
    ace_residue.add(acetyl_c1)
    ace_residue.add(acetyl_c2)
    ace_residue.add(acetyl_o)

    # Detach all residues to avoid index conflict
    for residue in l_residues:
        chain.detach_child(residue.id)

    # Insert the ACE residue
    chain.add(ace_residue)

    # Re-insert each residue with updated numbers
    for i, residue in enumerate(l_residues, start=2):
        residue.id = (residue.id[0], i, residue.id[2])
        chain.add(residue)

# Usage
add_acetyl(peptide5)

# Save the file
io = PDBIO()
io.set_structure(peptide5)
io.save("helical_6alanine_with_ACE.pdb")



Now you can look at that realistic acetil, placed realisticly in the N-terminus

In [33]:
#view
view = nv.show_file('helical_6alanine_with_ACE.pdb')
view.clear()
view.add_representation('ball+stick')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

Once again, the hydrogens are missing. Previously, we used pdb2gmx to solve this issue. But now there is a trap: we added a weird thing at the N-terminus, which will confuse the standard call of pdb2gmx. To avoid this trap, can use the option -ter to tell pdb2gmx not to treat the tips the way it would normally do. This option actually serves to choose how to treat the N and C termini of the peptide. In the case of the C terminus, we have to choose option 3 (None). This is the option that will save us from the trap.

Additionally, we will also be asked how to treat the C-terminus. The option 0 would add the standard COO- group. But let's use this opportunity to also add an N-terminus cap, so we just choose the option 2 (CT2). That intuitive name stands for the Amide cap, usually called NME. 

Notice how add a NME cap to the C-terminus is much easier than the manual construction and addition of the ACE to the N-terminus!

In [21]:
!printf '3\n3\n' | gmx pdb2gmx -f helical_6alanine_with_ACE.pdb -o helical_6alanine_with_ACE.gro -p helical_6alanine_with_ACE.top -i helical_6alanine_with_ACE.posres.itp -ignh -water tip3p -ff charmm27 -ter

               :-) GROMACS - gmx pdb2gmx, 2023.4-conda_forge (-:

Executable:   /home/hrigitano/miniconda3/envs/bio/bin.AVX2_256/gmx
Data prefix:  /home/hrigitano/miniconda3/envs/bio
Working dir:  /data3/henrique/templates/peptide_construction
Command line:
  gmx pdb2gmx -f helical_6alanine_with_ACE.pdb -o helical_6alanine_with_ACE.gro -p helical_6alanine_with_ACE.top -i helical_6alanine_with_ACE.posres.itp -ignh -water tip3p -ff charmm27 -ter

Using the Charmm27 force field in directory charmm27.ff

going to rename charmm27.ff/aminoacids.r2b
Opening force field file /home/hrigitano/miniconda3/envs/bio/share/gromacs/top/charmm27.ff/aminoacids.r2b

going to rename charmm27.ff/rna.r2b
Opening force field file /home/hrigitano/miniconda3/envs/bio/share/gromacs/top/charmm27.ff/rna.r2b
Reading helical_6alanine_with_ACE.pdb...
Read '', 33 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 7 residues with

Now we can take a look at our peptide. It should have all hydrogens, an ACE cap on the N-terminus, and an NME cap on the C-terminus.

In [22]:
#view
view = nv.show_file('helical_6alanine_with_ACE.gro')
view.clear()
view.add_representation('ball+stick')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

In [35]:
!cat helical_6alanine_with_ACE.gro

S  C  A  M  O  R  G
   72
    1ACE    CH3    1  -0.330   0.143   0.000
    1ACE   HH31    2  -0.327   0.043   0.000
    1ACE   HH32    3  -0.378   0.175   0.082
    1ACE   HH33    4  -0.378   0.175  -0.082
    1ACE      C    5  -0.190   0.197   0.000
    1ACE      O    6  -0.243   0.337   0.000
    2ALA      N    7  -0.052   0.136   0.000
    2ALA     HN    8   0.020   0.205   0.000
    2ALA     CA    9   0.000   0.000   0.000
    2ALA     HA   10  -0.034  -0.041   0.084
    2ALA     CB   11  -0.051  -0.077  -0.121
    2ALA    HB1   12  -0.014  -0.170  -0.119
    2ALA    HB2   13  -0.151  -0.081  -0.118
    2ALA    HB3   14  -0.021  -0.032  -0.204
    2ALA      C   15   0.152   0.000   0.000
    2ALA      O   16   0.214  -0.072   0.078
    3ALA      N   17   0.212   0.081  -0.087
    3ALA     HN   18   0.156   0.138  -0.147
    3ALA     CA   19   0.357   0.089  -0.096
    3ALA     HA   20   0.387  -0.003  -0.121
    3ALA     CB   21   0.399   0.189  -0.202
    3ALA    HB1   22   0.499 